In [1]:
from keras import layers

import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [2]:
IMG_SIZE = 128

In [6]:
center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)

def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped

def MostrarImagen(frame):
    cv2.imshow('Frame', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def IsBlack(frame, y, x):
    height, width, _ = frame.shape
    cond = True
    r = 2
    i = -2 * r
    j = -2 * r
    while i < r and i + y < height and i + y > 0 and cond:
        j = 0
        while j < r and j + x < width and j + x > 0 and cond:
            if frame[y+i, x+j].any() != 0:
                cond = False
            j = j + 1
        i = i + 1
              
    return cond

def EncontrarCorte(frame):
    height, width, _ = frame.shape

    halfWidth = int(width / 2)
    halfHeight = int(height / 2)

    up = 0
    down = height - 1
    left = halfWidth - 1
    right = halfWidth + 1

    while IsBlack(frame, up, halfWidth) and up < halfHeight:
            up = up + 1
    while IsBlack(frame, down, halfWidth) and down > halfHeight:
            down = down - 1

    while not IsBlack(frame, up + 8, left) and left > 0:
            left = left - 1
    while not IsBlack(frame, up + 8, right) and right < width:
            right = right + 1

    up = up - 1
    down = down + 1
    left = left - 1
    right = right + 1

    return right, up, left, down
    
def CalcularDimension(path):
    cap = cv2.VideoCapture(path)
    try:
            ret, frame = cap.read()

            right, up, left, down = EncontrarCorte(frame)
            MostrarImagen(frame[up:down,left:right])
            
            #nombre = path.split('\\')[1]
            #nombre = nombre + '_' + path.split('\\')[2].split('.')[0]
            #nombre = nombre.split('.') + '_' + path.split('\\')[2].split('.')[0]
            #cv2.imwrite('\\' + nombre +'.jpg', frame[up:down,left:right])
    finally:
        cap.release()
    return [right, up, left, down]
    
#CalcularDimension('Pruebas2020\\2.7.avi')

In [11]:
#dir = '..//Noviembre2020'
#dir = '..//Pruebas2020'
dir = '..//Pruebas2021'

df = pd.DataFrame({'Direccion':[], 'Dimension':[], 'Target':[]})

def MostrarFuncion(df, directorio):
    for archivo in os.listdir(directorio):

        ruta_completa = os.path.join(directorio, archivo)
        
        if os.path.isdir(ruta_completa):
            df = MostrarFuncion(df, ruta_completa)
        if os.path.isfile(ruta_completa) and (ruta_completa.split('.')[-1] == 'avi' or ruta_completa.split('.')[-1] == 'mov'):
            nueva_fila = {'Direccion':ruta_completa, 'Dimension':CalcularDimension(ruta_completa), 'Target' : 3}
            df = df.append(nueva_fila, ignore_index=True)

    return df

display(df)
df = MostrarFuncion(df, dir)
df.to_csv('../video list/lista_videos.csv', index=False)
#df.to_csv('lista_videos_especiales.csv', index=False)
#df.to_csv('lista_videos.csv', index=False)

,Direccion,Dimension,Target


C:\Users\Francisco\AppData\Local\Temp\ipykernel_5224\1721309896.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


In [4]:
df = pd.DataFrame({'Url':[], 'Label':[]})

def CreateCSV(df, root, label):
    for file in os.listdir(root):

        complete_url = os.path.join(root, file)
        
        if os.path.isdir(complete_url):
            df = CreateCSV(df, complete_url)
        if os.path.isfile(complete_url):# and (ruta_completa.split('.')[-1] == 'avi' or ruta_completa.split('.')[-1] == 'mov'):
            row = {'Url':complete_url, 'Label' : label}
            df = df.append(row, ignore_index=True)

    return df

display(df)
df = CreateCSV(df, '../Dataset/Frames/Normal/', 0)
df = CreateCSV(df, '../Dataset/Frames/B1/', 1)
df = CreateCSV(df, '../Dataset/Frames/B2/', 2)
df = CreateCSV(df, '../Dataset/Frames/C/', 3)

df.to_csv('../video list/frames_list.csv', index=False)

,Url,Label


C:\Users\Francisco\AppData\Local\Temp\ipykernel_7916\1988604299.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_7916\1988604299.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_7916\1988604299.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_7916\1988604299.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Francisco\A

In [3]:
#dir = '../Features/DenseNet121/'
dir = '../Features/ResNet101V2/'

def CambiarNombre(directorio):

    for archivo in os.listdir(directorio):
        ruta_completa = os.path.join(directorio, archivo)
        
        if os.path.isdir(ruta_completa):
            CambiarNombre(ruta_completa)
        if os.path.isfile(ruta_completa) and (ruta_completa.split('.')[-1] == 'npy'):
            nombre_archivo = os.path.basename(ruta_completa)
            corte = nombre_archivo.split('_')
            if (corte[0] == 'train' or corte[0] == 'test'):
                print(ruta_completa)
                print(corte)
                os.rename(ruta_completa, os.path.dirname(ruta_completa)+ '/' + corte[-1])

CambiarNombre(dir)

../Features/InceptionV3/PAC01\POST\train_data.npy
['train', 'data.npy']
../Features/InceptionV3/PAC01\POST\train_label.npy
['train', 'label.npy']
../Features/InceptionV3/PAC01\PRE\train_data.npy
['train', 'data.npy']
../Features/InceptionV3/PAC01\PRE\train_label.npy
['train', 'label.npy']
../Features/InceptionV3/PAC02\PD.BASAL.R.POST\train_data.npy
['train', 'data.npy']
../Features/InceptionV3/PAC02\PD.BASAL.R.POST\train_label.npy
['train', 'label.npy']
../Features/InceptionV3/PAC02\PD.FINAL.RPOST\train_data.npy
['train', 'data.npy']
../Features/InceptionV3/PAC02\PD.FINAL.RPOST\train_label.npy
['train', 'label.npy']
../Features/InceptionV3/PAC03\PD.BASAL.R.POST\train_data.npy
['train', 'data.npy']
../Features/InceptionV3/PAC03\PD.BASAL.R.POST\train_label.npy
['train', 'label.npy']
../Features/InceptionV3/PAC03\PD.FINAL.RPOST\test_data.npy
['test', 'data.npy']
../Features/InceptionV3/PAC03\PD.FINAL.RPOST\test_label.npy
['test', 'label.npy']
../Features/InceptionV3/PAC04\PD.supino\train_